In [ ]:
import numpy as np
import pandas as pd
import re
import math
from sklearn.metrics.pairwise import cosine_similarity
import heapq

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Skincare Product Dataset/result.csv")
df

,label,url,brand,name,price,skin type,spf,concern,concern 2,concern 3,key ingredient,formulation
0,face-moisturisers,https://www.myntra.com/face-moisturisers/lakme...,Lakme,Absolute Perfect Radiance Skin Lightening Day ...,₹ 79,All,NaN,General Care,NaN,NaN,NaN,Cream
1,face-moisturisers,https://www.myntra.com/face-moisturisers/bioti...,Biotique,Bio Morning Nectar Flawless Sustainable Skin M...,₹ 165,All,NaN,Uneven Skin Tone,Hydration,Dark Spots,Honey,Lotion
2,face-moisturisers,https://www.myntra.com/face-moisturisers/nivea...,Nivea,Unisex Aloe Hydration Skin Cream 100 ml,₹ 92,All,NaN,Dull Skin,Dryness,General Care,Aloe Vera,Cream
3,face-moisturisers,https://www.myntra.com/face-moisturisers/vi-jo...,VI-JOHN,Women Set of 5 Saffron Fairness Cream Advanced,₹ 187,All,15 to 30,Softening and Smoothening,Softening and Smoothening,Sun Protection,Vitamin E,Cream
4,face-moisturisers,https://www.myntra.com/face-moisturisers/lakme...,Lakme,Peach Milk Soft Creme 150g,₹ 192,All,NaN,General Care,NaN,NaN,NaN,Cream
...,...,...,...,...,...,...,...,...,...,...,...,...
1867,eye-cream,https://www.myntra.com/eye-cream/skinkraft/ski...,SKINKRAFT,Vitamin C+E Under Eye Gel for Pigmented Under ...,₹ 449,All,NaN,Dark Circles,NaN,NaN,Vitamin E,Gel
1868,eye-cream,https://www.myntra.com/eye-cream/casmara/casma...,CASMARA,Eye-Cream 15 ml,₹ 2200,All,NaN,Fine Lines and Wrinkles,Dark Circles,NaN,NaN,Gel
1869,eye-cream,https://www.myntra.com/eye-cream/myglamm/mygla...,MyGlamm,White Youthfull Hydrating Eye Cream With Water...,₹ 716,All,NaN,Dark Circles,Dark Circles,Eye Bags,Vitamin C,Cream
1870,eye-cream,https://www.myntra.com/eye-cream/makeup-revolu...,Makeup Revolution London,Hydro Bank Hydrating & Cooling Eye Balm,₹ 1050,All,NaN,NaN,NaN,NaN,NaN,Gel


In [ ]:
df['label'].value_counts()

,count
label,
face-moisturisers,300
cleanser,300
concealer,300
foundation,300
mask-and-peel,300
sunscreen,272
eye-cream,100


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1872 entries, 0 to 1871
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   label           1872 non-null   object
 1   url             1872 non-null   object
 2   brand           1826 non-null   object
 3   name            1826 non-null   object
 4   price           1826 non-null   object
 5   skin type       1563 non-null   object
 6   spf             420 non-null    object
 7   concern         1492 non-null   object
 8   concern 2       736 non-null    object
 9   concern 3       553 non-null    object
 10  key ingredient  713 non-null    object
 11  formulation     1488 non-null   object
dtypes: object(12)
memory usage: 175.6+ KB


In [ ]:
df.isna().sum()

,0
label,0
url,0
brand,46
name,46
price,46
skin type,309
spf,1452
concern,380
concern 2,1136
concern 3,1319


In [ ]:
df['concern 2'] = df['concern 2'].fillna('')
df['concern 3'] = df['concern 3'].fillna('')
df['concern'] = df['concern'] + ',' + df['concern 2'] + ',' + df['concern 3']
df['concern'] = df['concern'].str.replace(r',,+', ',', regex=True)

In [ ]:
df['concern']

,concern
0,"General Care,"
1,"Uneven Skin Tone,Hydration,Dark Spots"
2,"Dull Skin,Dryness,General Care"
3,"Softening and Smoothening,Softening and Smooth..."
4,"General Care,"
...,...
1867,"Dark Circles,"
1868,"Fine Lines and Wrinkles,Dark Circles,"
1869,"Dark Circles,Dark Circles,Eye Bags"
1870,NaN


In [ ]:
df.drop(columns=['concern 2', 'concern 3', 'spf', 'key ingredient', 'formulation', 'url', 'price'], inplace=True)
df['concern'].value_counts()

,count
concern,
"Sun Protection,",202
"Dark Spots,",81
"General Care,",68
"Dark Circles,",51
"Fine Lines and Wrinkles,",43
...,...
"Daily Use,Dryness,Dull Skin",1
"Dark Circles,Dark Circles,",1
"Fine Lines and Wrinkles,Dryness,Eye Bags",1


In [ ]:
df2 = df[((df['label'] == 'face-moisturisers') | (df['label'] == 'mask-and-peel') | (df['label'] == 'cleanser') | (df['label'] == 'sunscreen') |(df['label'] == 'eye-cream'))]
df2
LABELS = list(df2.label.unique())

In [ ]:
df2 = df2[df2['skin type'].isna() == False]
df2.index = [i for i in range(0, len(df2))]
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1229 entries, 0 to 1228
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   label      1229 non-null   object
 1   brand      1229 non-null   object
 2   name       1229 non-null   object
 3   skin type  1229 non-null   object
 4   concern    1197 non-null   object
dtypes: object(5)
memory usage: 57.6+ KB


In [ ]:
df2[df2['concern'].isna() == True]['label'].value_counts()

,count
label,
face-moisturisers,10
mask-and-peel,9
cleanser,5
sunscreen,4
eye-cream,4


In [ ]:
df2.loc[:, 'concern'] = df2['concern'].str.lower()
df2[df2['label'] == 'face-moisturisers']['concern'].value_counts()

,count
concern,
"general care,",35
"fine lines and wrinkles,",25
"pigmentation,",8
"softening and smoothening,",8
"uneven skin tone,hydration,dark spots",7
...,...
"fine lines and wrinkles,skin sagging",1
"hydration,deep nourishment,general care",1
"general care,softening and smoothening,pore care",1


In [ ]:
df2[df2['label'] == 'mask-and-peel']['concern'].value_counts()

,count
concern,
"daily use,",28
"acne or blemishes,",13
"hydration,deep nourishment,daily use",11
"oily skin,",9
"dull skin,hydration,daily use",8
...,...
"softening and smoothening,dryness,pore care",1
"daily use,dark spots,hydration",1
"hydration,hydration,softening and smoothening",1


In [ ]:
df2[df2['label'] == 'cleanser']['concern'].value_counts()

,count
concern,
"general care,",33
"acne or blemishes,",12
"dull skin,uneven skin tone,daily use",9
"anti-pollution,pore care,excess oil",7
"general care,daily use,",6
...,...
"general care,hydration,pore care",1
"daily use,excess oil,daily use",1
"dryness,dryness,softening and smoothening",1


In [ ]:
df2[df2['label'] == 'sunscreen']['concern'].value_counts()

,count
concern,
"sun protection,",195
"sun protection,dullness,redness",6
"sun protection,anti-pollution,",6
"sun protection,anti-pollution,tan removal",6
"sun protection,dullness,",6
"sun protection,dullness,tan removal",6
"sun protection,uneven skin tone,",4
"sun protection,tan removal,uneven skin tone",3
"sun protection,tan removal,",3


In [ ]:
df2[df2['label'] ==  'eye-cream']['concern'].value_counts()

,count
concern,
"fine lines and wrinkles,dark circles,eye bags",16
"dark circles,eye bags,",12
"dark circles,",11
"fine lines and wrinkles,dark circles,",9
"dark circles,dark circles,eye bags",8
"dark circles,eye bags,dryness",7
"dark circles,eye bags,dark circles",6
"dryness,dark circles,",3
"fine lines and wrinkles,",3


In [ ]:
top_concerns = {
    'face-moisturisers':'general care',
    'mask-and-peel':'daily use',
    'cleanser':'general care',
    'sunscreen':'sun protection',
    'eye-cream':'dark circles'
}

entries = len(df2)
for i in range(entries):
    label = df2.loc[i, 'label']
    if pd.isnull(df2.loc[i, 'concern']):  # cek apa concern kosong
        df2.loc[i, 'concern'] = top_concerns.get(label, '')  # mengisi concern dengan nilai top concern

df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1229 entries, 0 to 1228
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   label      1229 non-null   object
 1   brand      1229 non-null   object
 2   name       1229 non-null   object
 3   skin type  1229 non-null   object
 4   concern    1229 non-null   object
dtypes: object(5)
memory usage: 89.9+ KB


In [ ]:
df2

,label,brand,name,skin type,concern
0,face-moisturisers,Lakme,Absolute Perfect Radiance Skin Lightening Day ...,All,"general care,"
1,face-moisturisers,Biotique,Bio Morning Nectar Flawless Sustainable Skin M...,All,"uneven skin tone,hydration,dark spots"
2,face-moisturisers,Nivea,Unisex Aloe Hydration Skin Cream 100 ml,All,"dull skin,dryness,general care"
3,face-moisturisers,VI-JOHN,Women Set of 5 Saffron Fairness Cream Advanced,All,"softening and smoothening,softening and smooth..."
4,face-moisturisers,Lakme,Peach Milk Soft Creme 150g,All,"general care,"
...,...,...,...,...,...
1224,eye-cream,SKINKRAFT,Vitamin C+E Under Eye Gel for Pigmented Under ...,All,"dark circles,"
1225,eye-cream,CASMARA,Eye-Cream 15 ml,All,"fine lines and wrinkles,dark circles,"
1226,eye-cream,MyGlamm,White Youthfull Hydrating Eye Cream With Water...,All,"dark circles,dark circles,eye bags"
1227,eye-cream,Makeup Revolution London,Hydro Bank Hydrating & Cooling Eye Balm,All,dark circles


In [ ]:
df2.loc[:, 'brand'] = df2['brand'].str.lower()
df2.loc[:, 'name'] = df2['name'].str.lower()
df2.loc[:, 'skin type'] = df2['skin type'].str.lower()
df2.loc[:, 'concern'] = df2['concern'].str.replace(' and ', ',').str.replace(' or ', ',')
df2

,label,brand,name,skin type,concern
0,face-moisturisers,lakme,absolute perfect radiance skin lightening day ...,all,"general care,"
1,face-moisturisers,biotique,bio morning nectar flawless sustainable skin m...,all,"uneven skin tone,hydration,dark spots"
2,face-moisturisers,nivea,unisex aloe hydration skin cream 100 ml,all,"dull skin,dryness,general care"
3,face-moisturisers,vi-john,women set of 5 saffron fairness cream advanced,all,"softening,smoothening,softening,smoothening,su..."
4,face-moisturisers,lakme,peach milk soft creme 150g,all,"general care,"
...,...,...,...,...,...
1224,eye-cream,skinkraft,vitamin c+e under eye gel for pigmented under ...,all,"dark circles,"
1225,eye-cream,casmara,eye-cream 15 ml,all,"fine lines,wrinkles,dark circles,"
1226,eye-cream,myglamm,white youthfull hydrating eye cream with water...,all,"dark circles,dark circles,eye bags"
1227,eye-cream,makeup revolution london,hydro bank hydrating & cooling eye balm,all,dark circles


In [ ]:
def concern_elements(comma_sep_concerns):
    words = comma_sep_concerns.split(',')
    for w in words:
        if w != '':
            temp = w.rstrip()
            if temp in concerns:
                concerns[temp] += 1
            else:
                concerns[temp] = 1

list(df2['skin type'].unique())
concerns = {}
for i in range(entries):
    concern_elements(df2.iloc[i]['concern'])

In [ ]:
print(sorted(concerns.items(), key = lambda kv:(kv[1], kv[0])))

[('anti acne scarring', 1), ('anti-ageing', 2), ('skin tightening', 2), ('blackheads removal', 4), ('irregular textures', 6), ('skin inflammation', 11), ('oily skin', 16), ('skin sagging', 20), ('redness', 25), ('dullness', 26), ('lightening', 27), ('oil control', 28), ('eye bags', 56), ('blackheads', 61), ('whiteheads', 61), ('dark spots', 71), ('pigmentation', 71), ('tan removal', 72), ('anti-pollution', 79), ('excess oil', 90), ('uneven skin tone', 90), ('dark circles', 103), ('deep nourishment', 106), ('pore care', 110), ('smoothening', 112), ('softening', 112), ('dryness', 119), ('fine lines', 129), ('wrinkles', 129), ('acne', 155), ('blemishes', 155), ('hydration', 156), ('daily use', 158), ('general care', 158), ('dull skin', 166), ('sun protection', 279)]


In [ ]:
features = list(df2['skin type'].unique()) + list(concerns)
(features)

features = ['normal','dry','oily','combination', 'acne','sensitive','fine lines', 'wrinkles', 'redness', 'dull', 'pore', 'pigmentation', 'blackheads','whiteheads', 'blemishes', 'dark circles', 'eye bags', 'dark spots']

In [ ]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1229 entries, 0 to 1228
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   label      1229 non-null   object
 1   brand      1229 non-null   object
 2   name       1229 non-null   object
 3   skin type  1229 non-null   object
 4   concern    1229 non-null   object
dtypes: object(5)
memory usage: 89.9+ KB


In [ ]:
len(features)

18

In [ ]:
def search_concern(target, i):
    if target in df2.iloc[i]['concern']:
        return True
    return False

one_hot_encodings = np.zeros([entries, len(features)])

#skin types first
for i in range(entries):
    for j in range(5):
        target = features[j]
        sk_type = df2.iloc[i]['skin type']
        if sk_type == 'all':
            one_hot_encodings[i][0:5] = 1
        elif target == sk_type:
            one_hot_encodings[i][j] = 1

#other features
for i in range(entries):
    for j in range(5, len(features)):
        feature = features[j]
        if feature in df2.iloc[i]['concern']:
            one_hot_encodings[i][j] = 1

In [ ]:
x = one_hot_encodings[456]

In [ ]:
from sklearn.neighbors import NearestNeighbors
nbrs = NearestNeighbors(n_neighbors=6, algorithm='ball_tree').fit(one_hot_encodings)
distances, indices = nbrs.kneighbors(one_hot_encodings)

In [ ]:
# cari poin data terdekat
def recs_nn(query=None,id=None):
    if id:
        for id in indices[id][1:]:
            print(anime.ix[id]["name"])
    if query:
        found_id = get_index_from_name(query)
        for id in indices[found_id][1:]:
            print(index2prod(id))

Cosine Similarity

In [ ]:
def name2index(name):
    return df2[df2["name"]==name].index.tolist()[0]

def index2prod(index):
    return df2.iloc[index]

In [ ]:
def wrap(info_arr):
    result = {}
    result['brand'] = info_arr[0]
    result['name'] = info_arr[1]
    result['skin type'] = info_arr[2]
    result['concern'] = str(info_arr[3]).split(',')
    return result

def recs_cs(vector=None, name=None, label=None, count=5):
    products = []
    if name:
        idx = name2index(name)
        fv = one_hot_encodings[idx]
    elif vector:
        fv = vector
    cs_values = cosine_similarity(np.array([fv, ]), one_hot_encodings)
    df2['cs'] = cs_values[0]

    if label:
        dff = df2[df2['label'] == label]
    else:
        dff = df2

    if name:
        dff = dff[dff['name'] != name]
    recommendations = dff.sort_values('cs', ascending=False).head(count)
    data = recommendations[['brand', 'name', 'skin type', 'concern']].to_dict('split')['data']
    for element in data:
        products.append(wrap(element))
    return products

In [ ]:
def recs_essentials(vector = None, name = None):
#     print("ESSENTIALS:")
    response = {}
    for label in LABELS:
#         print(f"{label}:")
        if name:
            r = recs_cs(None, name, label)
        elif vector:
            r = recs_cs(vector, None, label)
        response[label] = r
    return response

In [ ]:
def display_recommendations(recommendations):
    for category, products in recommendations.items():
        print(f"\n {category.capitalize()}:")
        for idx, product in enumerate(products, start=1):
            print(f"{idx}. Brand: {product['brand']}")
            print(f"   Nama Produk: {product['name']}")
            print(f"   Tipe Kulit: {product['skin type']}")
            print(f"   Concern: {', '.join(product['concern'])}\n")

In [ ]:
def get_user_input(features):
    print("Skin Type yang dimiliki: Normal")
    print("Concern yang dimiliki: Acne")
    print("Daftar jenis kulit (skin type dan concerns):")
    for idx, feature in enumerate(features):
        print(f"{idx}: {feature}")

    user_input = input("Masukkan tambahan permasalahan lain yang anda alami sebagai list angka (contoh: 0, 2, 5): ")
    concern_indices = list(map(int, user_input.split(',')))

    user_vector = [1 if i in concern_indices else 0 for i in range(len(features))]
    return user_vector

user_vector = get_user_input(features)
recommendations = recs_essentials(user_vector, None)
display_recommendations(recommendations)

Skin Type yang dimiliki: Normal
Concern yang dimiliki: Acne
Daftar jenis kulit (skin type dan concerns):
0: normal
1: dry
2: oily
3: combination
4: acne
5: sensitive
6: fine lines
7: wrinkles
8: redness
9: dull
10: pore
11: pigmentation
12: blackheads
13: whiteheads
14: blemishes
15: dark circles
16: eye bags
17: dark spots
Masukkan tambahan permasalahan lain yang anda alami sebagai list angka (contoh: 0, 2, 5): 17

 Face-moisturisers:
1. Brand: lakme
   Nama Produk: 9 to 5 matte moist mattifying moisturiser with green tea- 50 g
   Tipe Kulit: normal
   Concern: general care, dark spots, excess oil

2. Brand: kama ayurveda
   Nama Produk: sustainable rejuvenating brightening ayurvedic night cream 25g
   Tipe Kulit: normal
   Concern: pigmentation, dark spots, softening, smoothening

3. Brand: mamaearth
   Nama Produk: vitamin c face cream with spf 20 50g
   Tipe Kulit: normal
   Concern: dull skin, dark spots, 

4. Brand: phy
   Nama Produk: mens green tea superlight moisturizer
   Tip